In [23]:
import json
import pandas as pd
import os
import numpy as np

with open('../nflscraping/json/espn_scores.json') as rosters:
    data = json.load(rosters)

df = pd.json_normalize(data)
df

,season,week,awayteam,hometeam,awayscore,homescore,gamecast,boxscore,idgame,awayteam global record,awayteam away record,hometeam global record,hometeam home record
0,2020,1,Texans,Chiefs,20,34,https://espn.com/nfl/game/_/gameId/401220225,https://espn.com/nfl/boxscore/_/gameId/401220225,401220225,0-1,0-1,1-0,1-0
1,2020,1,Seahawks,Falcons,38,25,https://espn.com/nfl/game/_/gameId/401220313,https://espn.com/nfl/boxscore/_/gameId/401220313,401220313,1-0,1-0,0-1,0-1
2,2020,1,Jets,Bills,17,27,https://espn.com/nfl/game/_/gameId/401220116,https://espn.com/nfl/boxscore/_/gameId/401220116,401220116,0-1,0-1,1-0,1-0
3,2020,1,Bears,Lions,27,23,https://espn.com/nfl/game/_/gameId/401220282,https://espn.com/nfl/boxscore/_/gameId/401220282,401220282,1-0,1-0,0-1,0-1
4,2020,1,Packers,Vikings,43,34,https://espn.com/nfl/game/_/gameId/401220300,https://espn.com/nfl/boxscore/_/gameId/401220300,401220300,1-0,1-0,0-1,0-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1289,2021,18,49ers,Rams,27,24,https://espn.com/nfl/game/_/gameId/401326599,https://espn.com/nfl/boxscore/_/gameId/401326599,401326599,10-7,6-3,12-5,5-3
1290,2021,18,Patriots,Dolphins,24,33,https://espn.com/nfl/game/_/gameId/401326592,https://espn.com/nfl/boxscore/_/gameId/401326592,401326592,10-7,6-2,9-8,6-3
1291,2021,18,Seahawks,Cardinals,38,30,https://espn.com/nfl/game/_/gameId/401326597,https://espn.com/nfl/boxscore/_/gameId/401326597,401326597,7-10,4-5,11-6,3-5
1292,2021,18,Panthers,Buccaneers,17,41,https://espn.com/nfl/game/_/gameId/401326596,https://espn.com/nfl/boxscore/_/gameId/401326596,401326596,5-12,3-6,13-4,7-1


In [24]:
df.drop(['boxscore', 'gamecast',"awayteam global record","awayteam away record","hometeam global record","hometeam home record"], axis=1, inplace=True)
df.reset_index()
df

,season,week,awayteam,hometeam,awayscore,homescore,idgame
0,2020,1,Texans,Chiefs,20,34,401220225
1,2020,1,Seahawks,Falcons,38,25,401220313
2,2020,1,Jets,Bills,17,27,401220116
3,2020,1,Bears,Lions,27,23,401220282
4,2020,1,Packers,Vikings,43,34,401220300
...,...,...,...,...,...,...,...
1289,2021,18,49ers,Rams,27,24,401326599
1290,2021,18,Patriots,Dolphins,24,33,401326592
1291,2021,18,Seahawks,Cardinals,38,30,401326597
1292,2021,18,Panthers,Buccaneers,17,41,401326596


In [30]:
#df = df.loc[((df.awayteam=='Texans') | (df.hometeam=='Texans')) & (df.season=='2020')]
#from calendar import week


df.week = df.week.astype(int)
df.awayscore = df.awayscore.astype(int)
df.homescore = df.homescore.astype(int)
df.season = df.season.astype(int)
df = df.sort_values(by=['season', 'week'],ascending = [True, True])
#   - Addition of analysis features:

#df.insert(df.shape[1],"score_abs",df["homescore"] - df["awayscore"]) #Absolute score of the game
df = df.assign(scoreabs=df.awayscore-df.homescore)
df = df.assign(winner=df.hometeam)
df= df.assign(loser=df.awayteam)
df.winner = df[['awayteam','hometeam','scoreabs']].apply(lambda x: x['awayteam'] if x['scoreabs']>0 else x['hometeam'], axis=1)
df.loser = df[['awayteam','hometeam','scoreabs']].apply(lambda x: x['awayteam'] if x['scoreabs']<0 else x['hometeam'], axis=1)

#df = df.assign(streak_home=0)
#df = df.assign(streak_away=0)
#df.streak_home = df[['awayteam','hometeam','winner']].apply(lambda x: 1 if x['hometeam']==x['winner'] else 0, axis=1)
#df.streak_away = df[['awayteam','hometeam','winner']].apply(lambda x: 1 if x['awayteam']==x['winner'] else 0, axis=1)

df_teams_stats_away = pd.DataFrame().assign(season=df.season, week=df.week, idgame=df.idgame, team=df.awayteam, winner=df.winner, loser=df.loser)
df_teams_stats_home = pd.DataFrame().assign(season=df.season, week=df.week, idgame=df.idgame, team=df.hometeam, winner=df.winner, loser=df.loser)

#df_teams_stats.append({'season' : df.season, 'week' : df.week, 'idgame' : df.idgame, 'team' : df.hometeam, 'winner' : df.winner},ignore_index=True)

def f(x):
    x['streak2'] = (x['winner'] == x['team']).cumsum()
    x['cumsum'] = np.nan
    #x.loc[x['winner'] != x['team'], 'cumsum'] = df['streak2']
    #x['cumsum'] = x['cumsum'].fillna(method='ffill')
    #x['cumsum'] = x['cumsum'].fillna(0)
    #x['streak_after'] = x['streak2'] - x['cumsum']
    #x.drop(['streak2', 'cumsum'], axis=1, inplace=True)
    return x

#d.concat([new_df,entry])
df_teams_stats = pd.concat([df_teams_stats_away,df_teams_stats_home], ignore_index=True, sort=False)
df_teams_stats = df_teams_stats.sort_values(by=['season','team','week'],ascending=[True,True,True])
df_teams_stats = df_teams_stats.reset_index(drop=True,inplace=False)
df_teams_stats = df_teams_stats.assign(streak_before=0)
df_teams_stats = df_teams_stats.assign(streak_after=0)
df_teams_stats.streak_before = df_teams_stats[['team','winner','week']].apply(lambda x: 0 if (x['week']==1) else -1, axis=1)
df_teams_stats.streak_after = df_teams_stats[['team','winner','week']].apply(lambda x: 1 if (x['week']==1 & (x['winner']==x['team'])) else -1, axis=1)
df_teams_stats = df_teams_stats.groupby('winner').apply(f)
df_teams_stats = df_teams_stats.loc[df_teams_stats.team=='Washington']
df_teams_stats



#df_teams_stats = df_teams_stats.sort_values(by=['season','week','idgame'],ascending=[True,True,True])
#df_teams_stats = df_teams_stats.reset_index(drop=True,inplace=True)
#df_teams_stats
#df_teams_stats = df.assign(streak=0)
#df_teams_stats.streak = df_teams_stats[['team','winner','week']].apply(lambda x: 1 if (x['team']==x['winner']) else 0, axis=1)
#df_team_stats = df_teams_stats.groupby(['idgame'])
#series = df.iloc[:,7]
#df['streak'] = df.result.ne(df['winner'].shift())
#df.streak = df[['awayteam','hometeam','scoreabs','week','streak']].apply(lambda x: x['streak']+1 if (x['scoreabs']>0 else x['hometeam'], axis=1)
#df.insert(df.shape[1],"winner_home", 0) #Flagging whether the home team won (1) or lose (0)

#df.loc[:,['streak']] = df.loc[(df.winner & df.week-1)]

#def f(x):
    #x.past_week = x.winner
    #x.streak = x.groupby( (x['stat'] != 0).cumsum()).cumcount()+( (x['stat'] != 0).cumsum() == 0).astype(int) 

#df_teams_stats_home

#df.loc[df.season==2017]
#df.loc[(df.player_name=='Kyler Murray') & (df.game_id==401326597)]

KeyError: 'streak2'

In [26]:
def f(x):
    df.streak = x.groupby( (x['stat'] != 0).cumsum()).cumcount() + ( (x['stat'] != 0).cumsum() == 0).astype(int) 
    return x

In [27]:

for i in range(len(df["winner_home"])):
    if df["score_abs"][i] > 0 :
        df["winner_home"][i] = 1
    elif df["score_abs"][i] < 0 :
        df["winner_home"][i] = 0
    else :
        df["winner_home"][i] = 999999999 #Value to filter and remove later on as values are not relevant vs amount of data


df.insert(df.shape[1],"winner_away", 0) #Flagging whether the away team won (1) or lose (0)

for i in range(len(df["winner_away"])):
    if df["score_abs"][i] < 0 :
        df["winner_away"][i] = 1
    elif df["score_abs"][i] > 0 :
        df["winner_away"][i] = 0
    else :
        df["winner_away"][i] = 999999999 #Value to filter and remove later on as values are not relevant vs amount of data

df

KeyError: 'winner_home'

In [ ]:
#df['homestreak'] = 
'''
df['week'] = df['week'].astype(int)
df['awayscore'] = df['awayscore'].astype(float)
df['homescore'] = df['homescore'].astype(float)
df = df.loc[((df.awayteam=='Texans') | (df.hometeam=='Texans')) & (df.season=='2020')]
df.sort_values(by = 'week')
df.reset_index()

df.drop(['boxscore', 'idgame', 'gamecast', "awayteam global record", "awayteam away record", "hometeam global record", "hometeam home record"], axis=1, inplace=True)

#df.info()
df


for index, row in df.iterrows():
  if(df['awayscore']<<df['homescore']):
    df['homestreak'] = -1
    df['awaystreak'] = 1

#for index,row in df.iterrows():
#for i in df:
  #if df[]
  #if df.at[index,'week']==1:


    #if(df['awayscore']<<df['homescore']):
    #  df['homestreak'] = -1
    #  df['awaystreak'] = 1
  #elif df.at[index,'week']==2:
    #df.loc[(df.)]
    #df.at[index,'streak'] = '83'
    #df.at[index,'streak'] = d
  #else:
  #  df['homestreak'] = 0
  #  df['awaystreak'] = 0

#for wk in df:
'''